In [ ]:
import os
os.chdir("..")
import sys
sys.path.append("clear/pytorch")

Download [BER dataset](https://ndber.seai.ie/BERResearchTool/ber/search.aspx)

Put the dataset under the input folder (./input)

In [21]:
!unzip input/BERPublicSearch.zip -d input

Archive:  input/BERPublicSearch.zip
  inflating: input/BERPublicsearch.txt  ^C


In [ ]:
import pandas as pd
#file saved with encoding utf-8
data = pd.read_csv("input/BERPublicsearch.txt", 
                   sep='\t', 
                   header=0, 
                   encoding='latin_1',
                   on_bad_lines='warn'
                  )

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

# Split the data into train and test sets and stratify by the EnergyRating
stratified_sample, _ = train_test_split(data, test_size=0.9, stratify=data[['EnergyRating']])

In [ ]:
#save stratified sample to csv
stratified_sample.to_csv('input/ber-csv/BER_stratified_sample.csv')
#save full data to csv
data.to_csv("input/ber-csv/BER.csv")


In [9]:
import pandas as pd
raw_data_path = "input/ber-csv/BER_stratified_sample.csv"
X_raw = pd.read_csv(raw_data_path)

/var/folders/lj/fb1h6p1n6037k3t6r3ltp7fw0000gn/T/ipykernel_11763/775515106.py:3: DtypeWarning: Columns (156,162,167) have mixed types. Specify dtype option on import or set low_memory=False.
  X_raw = pd.read_csv(raw_data_path)


In [8]:
n_splits = 5
output_dir = "output"

In [ ]:
import os
def save_csv_file(df, path):
    """
    Save a DataFrame as a CSV file.

    Args:
        df (pandas.DataFrame): The DataFrame to be saved.
        path (str): The path where the CSV file will be saved.

    Returns:
        None
    """
    dir_name = os.path.dirname(path)
    if dir_name:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    df.to_csv(path, index=False)

In [ ]:
from sklearn.model_selection import KFold
import os
# Split the data into train and test sets using KFold cross-validation
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)

# Iterate over each fold and save the train and test data
for i, (train_index, test_index) in enumerate(kf.split(X_raw)):
    save_csv_file(X_raw.iloc[train_index], f"{output_dir}/split_{i+1}/raw_train.csv")
    save_csv_file(X_raw.iloc[test_index], f"{output_dir}/split_{i+1}/raw_test.csv")

In [13]:
#run preprocessing script
config_dir = "clear/pytorch/configs"
command = f"""
    python clear/pytorch/get_processed_dataset.py \
      --config_dir="{config_dir}" \
      --output_dir="{output_dir}" \
      --data_path={raw_data_path} \
      --output_csv_name "processed" \
      --is_train
    """
os.system(command)

0

In [15]:
#copy config files to each split
files_to_copy = ["column_type_classification.yaml", "encoder.joblib", "scaler.joblib", "train_stats.json"]
for i in range(n_splits):
    for file_name in files_to_copy:
        src = f"{output_dir}/{file_name}"
        dest = f"{output_dir}/split_{i+1}/{file_name}"
        os.system(f"cp {src} {dest}")

In [17]:
#run preprocessing script for each split train and test data
for i in range(n_splits):
    command = f"""
    python clear/pytorch/get_processed_dataset.py \
      --config_dir="{config_dir}" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_train.csv" \
      --output_csv_name "processed_train" \
    """
    os.system(command)
    command = f"""
    python clear/pytorch/get_processed_dataset.py \
      --config_dir="{config_dir}" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_test.csv" \
      --output_csv_name "processed_test"
    """
    os.system(command)

In [18]:
import os
def convert(df, filefolder, fname):
    """
    Convert a DataFrame to a parquet file.

    Parameters:
    - df: DataFrame
        The DataFrame to be converted.
    - filefolder: str
        The folder path where the parquet file will be saved.
    - fname: str
        The name of the parquet file (without extension) to save.

    Returns:
    None
    """
    os.makedirs(filefolder, exist_ok=True) 
    df.to_parquet(f"{filefolder}/{fname}.parquet", compression='gzip')

In [20]:
import pandas as pd
data_dir = "output"
output_dir = "output"
#save processed data in parquet format
processed = pd.read_csv(f"{data_dir}/processed.csv")
convert(processed, output_dir, "processed")
selected = range(n_splits)
for i in selected:
    train_file, test_file = "processed_train", "processed_test"
    train_df = pd.read_csv(f"{data_dir}/split_{i+1}/{train_file}.csv")
    convert(train_df, f"{output_dir}/split_{i+1}/", f"{train_file}")
    test_df = pd.read_csv(f"{data_dir}/split_{i+1}/{test_file}.csv")
    convert(test_df, f"{output_dir}/split_{i+1}/", f"{test_file}")